In [1]:
from pynq import Overlay
ol = Overlay("axi_stream.bit")

ol.download()
ol.bitstream.timestamp

from pynq import PL
PL.bitfile_name

ol.is_loaded()

True

In [2]:
import time
import math
from pynq import MMIO
axis = MMIO(0x40400000,0x00010000)
MM2S_CONTROL_REGISTER=0x00
MM2S_STATUS_REGISTER=0x04
MM2S_START_ADDRESS=0x18
MM2S_LENGTH=0x28

S2MM_CONTROL_REGISTER=0x30
S2MM_STATUS_REGISTER=0x34
S2MM_DESTINATION_ADDRESS=0x48
S2MM_LENGTH=0x58

In [3]:
print(hex(axis.read(MM2S_CONTROL_REGISTER)))
print(hex(axis.read(MM2S_STATUS_REGISTER)))
print(hex(axis.read(MM2S_START_ADDRESS)))
print(hex(axis.read(MM2S_LENGTH)))
print(hex(axis.read(S2MM_CONTROL_REGISTER)))
print(hex(axis.read(S2MM_STATUS_REGISTER)))
print(hex(axis.read(S2MM_DESTINATION_ADDRESS)))
print(hex(axis.read(S2MM_DESTINATION_ADDRESS)))

0x10002
0x10009
0x0
0x0
0x10002
0x10009
0x0
0x0


In [ ]:
def dma_status(REGISTER_OFFSET):
    global axis
    status=axis.read(REGISTER_OFFSET)
    if (REGISTER_OFFSET==0x34):
        print("Stream to memory-mapped status: "+str(hex(status))+"@"+str(REGISTER_OFFSET))  
    else: 
        print("Memory-mapped to stream status: "+str(hex(status))+"@"+str(REGISTER_OFFSET))  
    print("halted") if (status & 0x00000001) else print("running")
    if (status & 0x00000002): print("idle")  
    if (status & 0x00000008): print("SGIncld")  
    if (status & 0x00000010): print("DMAIntErr") 
    if (status & 0x00000020): print("DMASlvErr") 
    if (status & 0x00000040): print("DMADecErr")
    if (status & 0x00000100): print("SGIntErr") 
    if (status & 0x00000200): print("SGSlvErr") 
    if (status & 0x00000400): print("SGDecErr") 
    if (status & 0x00001000): print("IOC_Irq") 
    if (status & 0x00002000): print("Dly_Irq") 
    if (status & 0x00004000): print("Err_Irq") 
    #print("\n");

def dma_set(offset,value):
    global axis
    axis.write(offset,value)
    #print(hex(offset))

def dma_get(offset):
    global axis
    val=axis.read(offset)
    #print(hex(val))
    return val

def memdump(loc,byteCount):
    global axis
    for offset in range(0,byteCount):
        print("LOC ",str(offset), hex(axis.read(loc+offset*4)));
        if offset%4==3: print(" ") 
    print("\n") 

def dma_mm2s_sync():
    mm2s_status=dma_get(MM2S_STATUS_REGISTER)
    while(~(mm2s_status & 1<<12) | ~(mm2s_status & 1<<1) ):
        #dma_status(S2MM_STATUS_REGISTER)
        #dma_status(MM2S_STATUS_REGISTER)
        mm2s_status=dma_get(MM2S_STATUS_REGISTER)

def dma_s2mm_sync():
    s2mm_status=dma_get(S2MM_STATUS_REGISTER)
    while(~(mm2s_status & 1<<12) | ~(mm2s_status & 1<<1) ):
        #dma_status(S2MM_STATUS_REGISTER)
        #dma_status(MM2S_STATUS_REGISTER)
        s2mm_status=dma_get(S2MM_STATUS_REGISTER)

In [ ]:
source_address=0x0e000000
destination_address=0x0f000000
#print("Source memory block:      ")
#memdump(source_address,32);
#print("Destination memory block:      ")
#memdump(destination_address,32);

print("Resetting DMA")    
dma_set(S2MM_CONTROL_REGISTER, 4)
dma_set(MM2S_CONTROL_REGISTER, 4)
dma_status(S2MM_STATUS_REGISTER)
dma_status(MM2S_STATUS_REGISTER)

print("Halting DMA")   
dma_set(S2MM_CONTROL_REGISTER, 0);
dma_set(MM2S_CONTROL_REGISTER, 0);
dma_status(S2MM_STATUS_REGISTER);
dma_status(MM2S_STATUS_REGISTER);

print("Writing destination address..");
dma_set(S2MM_DESTINATION_ADDRESS, 0x0f000000);
dma_status(S2MM_STATUS_REGISTER);

print("Writing source address...");
dma_set(MM2S_START_ADDRESS, 0x0e000000); 
dma_status(MM2S_STATUS_REGISTER);

print("Starting S2MM channel with all interrupts masked...");
dma_set(S2MM_CONTROL_REGISTER, 0xf001);
dma_status(S2MM_STATUS_REGISTER);

print("Starting MM2S channel with all interrupts masked...");
dma_set(MM2S_CONTROL_REGISTER, 0xf001);
dma_status(MM2S_STATUS_REGISTER);

print("Writing S2MM transfer length...");
dma_set(S2MM_LENGTH, 32);
dma_status(S2MM_STATUS_REGISTER);

print("Writing MM2S transfer length...");
dma_set(MM2S_LENGTH, 32);
dma_status(MM2S_STATUS_REGISTER);

print("Waiting for MM2S synchronization..");
dma_mm2s_sync();

print("Waiting for S2MM sychronization...");
dma_s2mm_sync(); 

dma_status(S2MM_STATUS_REGISTER);
dma_status(MM2S_STATUS_REGISTER);


Resetting DMA
Stream to memory-mapped status: 0x10009@52
halted
SGIncld
Memory-mapped to stream status: 0x10009@4
halted
SGIncld
Halting DMA
Stream to memory-mapped status: 0x10009@52
halted
SGIncld
Memory-mapped to stream status: 0x10009@4
halted
SGIncld
Writing destination address..
Stream to memory-mapped status: 0x10009@52
halted
SGIncld
Writing source address...
Memory-mapped to stream status: 0x10009@4
halted
SGIncld
Starting S2MM channel with all interrupts masked...
Stream to memory-mapped status: 0x10008@52
running
SGIncld
Starting MM2S channel with all interrupts masked...
Memory-mapped to stream status: 0x10008@4
running
SGIncld
Writing S2MM transfer length...
Stream to memory-mapped status: 0x10008@52
running
SGIncld
Writing MM2S transfer length...
Memory-mapped to stream status: 0x10008@4
running
SGIncld
Waiting for MM2S synchronization..
